# HELP MATE AI

## Install all the required libraries

In [1]:
!pip install -U -q pdfplumber tiktoken openai chromaDB sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.7/767.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 4.8 MB/s eta

In [2]:
# Import all the required Libraries

import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import openai
import chromadb
import os
from google.colab import userdata
from sentence_transformers import CrossEncoder, util


In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
## validating if we are able to  conect and list data drives
!ls "/content/drive/MyDrive/Colab Notebooks/Principal-Sample-Life-Insurance-Policy.pdf"

'/content/drive/MyDrive/Colab Notebooks/Principal-Sample-Life-Insurance-Policy.pdf'


## Reading the PDF file

In [5]:
pdf_file_path ="/content/drive/MyDrive/Colab Notebooks/Principal-Sample-Life-Insurance-Policy.pdf"

In [6]:
# Open the PDF file
with pdfplumber.open(pdf_file_path) as pdf:

    # Get one of the pages from the PDF and examine it
    single_page = pdf.pages[2]

    # Extract text from the first page
    text = single_page.extract_text()

    # Extract tables from the first page
    tables = single_page.extract_tables()

    # Print the extracted text
    print(text)
    print(tables)

POLICY RIDER
GROUP INSURANCE
POLICY NO: S655
COVERAGE: Life
EMPLOYER: RHODE ISLAND JOHN DOE
Effective on the later of the Date of Issue of this Group Policy or March 1, 2005, the following
will apply to your Policy:
From time to time The Principal may offer or provide certain employer groups who apply
for coverage with The Principal a Financial Services Hotline and Grief Support Services or
any other value added service for the employees of that employer group. In addition, The
Principal may arrange for third party service providers (i.e., optometrists, health clubs), to
provide discounted goods and services to those employer groups who apply for coverage
with The Principal or who become insureds/enrollees of The Principal. While The
Principal has arranged these goods, services and/or third party provider discounts, the third
party service providers are liable to the applicants/insureds/enrollees for the provision of
such goods and/or services. The Principal is not responsible for the 

### Funtion to check if words boundries are inside table boundries. Returns True if word is inside table.

In [7]:
def is_word_inside_table(word, table_bbox):
    # Check whether word is inside a table bbox.
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

## Function to extract text from a PDF file.

In [8]:

# 1. Declare a variable p to store the iteration of the loop that will help us store page numbers alongside the text
# 2. Declare an empty list 'full_text' to store all the text files
# 3. Use pdfplumber to open the pdf pages one by one
# 4. Find the tables and their locations in the page
# 5. Extract the text from the tables in the variable 'tables'
# 6. Extract the regular words by calling the function check_bboxes() and checking whether words are present in the table or not
# 7. Use the cluster_objects utility to cluster non-table and table words together so that they retain the same chronology as in the original PDF
# 8. Declare an empty list 'lines' to store the page text
# 9. If a text element in present in the cluster, append it to 'lines', else if a table element is present, append the table
# 10. Append the page number and all lines to full_text, and increment 'p'
# 11. When the function has iterated over all pages, return the 'full_text' list

def extract_text_from_pdf(pdf_path):
    p = 0
    full_text = []


    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_no = f"Page {p+1}"
            text = page.extract_text()
            heading = text.split('\n')[0].strip() if text else None
            tables = page.find_tables()
            table_bboxes = [i.bbox for i in tables]
            tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
            non_table_words = [word for word in page.extract_words() if not any(
                [is_word_inside_table(word, table_bbox) for table_bbox in table_bboxes])]
            lines = []

            for cluster in pdfplumber.utils.cluster_objects(non_table_words + tables, itemgetter('top'), tolerance=5):

                if 'text' in cluster[0]:
                    try:
                        lines.append(' '.join([i['text'] for i in cluster]))
                    except KeyError:
                        pass

                elif 'table' in cluster[0]:
                    lines.append(json.dumps(cluster[0]['table']))


            full_text.append([page_no,heading, " ".join(lines)])
            p +=1

    return full_text

In [9]:
extracted_text = extract_text_from_pdf(pdf_file_path)

In [10]:
 # Convert the extracted list to a PDF, and add a column to store document names
extracted_text_df = pd.DataFrame(extracted_text, columns=['Page No.', 'Heading','Page_Text'])
extracted_text_df.head()

,Page No.,Heading,Page_Text
0,Page 1,DOROTHEA GLAUSE S655,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...
1,Page 2,This page left blank intentionally,This page left blank intentionally
2,Page 3,POLICY RIDER,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...
3,Page 4,This page left blank intentionally,This page left blank intentionally
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY,PRINCIPAL LIFE INSURANCE COMPANY (called The P...


In [11]:
extracted_text_df['Text_Length'] = extracted_text_df['Page_Text'].apply(lambda x: len(x.split(' ')))

In [12]:
extracted_text_df.head(10)

,Page No.,Heading,Page_Text,Text_Length
0,Page 1,DOROTHEA GLAUSE S655,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,30
1,Page 2,This page left blank intentionally,This page left blank intentionally,5
2,Page 3,POLICY RIDER,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,230
3,Page 4,This page left blank intentionally,This page left blank intentionally,5
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,110
5,Page 6,TABLE OF CONTENTS,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,153
6,Page 7,Section A – Eligibility,Section A – Eligibility Member Life Insurance ...,176
7,Page 8,Section A - Member Life Insurance,Section A - Member Life Insurance Schedule of ...,171
8,Page 9,P ART I - DEFINITIONS,P ART I - DEFINITIONS When used in this Group ...,387
9,Page 10,T he legally recognized union of two eligible ...,T he legally recognized union of two eligible ...,251


### Remove empty or blank pages

In [13]:
extracted_text_df = extracted_text_df.loc[extracted_text_df['Text_Length']>=10]
extracted_text_df.head(10)

,Page No.,Heading,Page_Text,Text_Length
0,Page 1,DOROTHEA GLAUSE S655,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,30
2,Page 3,POLICY RIDER,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,230
4,Page 5,PRINCIPAL LIFE INSURANCE COMPANY,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,110
5,Page 6,TABLE OF CONTENTS,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,153
6,Page 7,Section A – Eligibility,Section A – Eligibility Member Life Insurance ...,176
7,Page 8,Section A - Member Life Insurance,Section A - Member Life Insurance Schedule of ...,171
8,Page 9,P ART I - DEFINITIONS,P ART I - DEFINITIONS When used in this Group ...,387
9,Page 10,T he legally recognized union of two eligible ...,T he legally recognized union of two eligible ...,251
10,Page 11,(2) has been placed with the Member or spouse ...,(2) has been placed with the Member or spouse ...,299
11,Page 12,An institution that is licensed as a Hospital ...,An institution that is licensed as a Hospital ...,352


In [14]:
print(extracted_text_df['Heading'].str.len())

0     20
2     12
4     32
5     17
6     23
7     33
8     21
9     98
10    83
11    97
12    65
13    31
14    94
15    31
16    88
17    92
18    92
19    20
20    69
21    84
22    30
23    87
24    26
25    45
26    85
27    27
28    91
29    87
30    82
31    89
32    93
33    86
34    35
35    95
36    28
37    24
38    89
39    25
40    93
41    38
42    81
43    87
44    82
45    18
46    92
47    94
48    97
49    84
50    57
51    83
52    63
53    75
54     8
55    97
56    14
57    68
58    36
59    94
60    28
61    93
Name: Heading, dtype: int64


In [15]:
extracted_text_df['Metadata'] = extracted_text_df.apply(
    lambda x: {
        'Section': (x['Heading'][:25] if x['Heading'] else ''),
        'Page_No.': x['Page No.']
    },
    axis=1
)

In [16]:
extracted_text_df.reset_index(drop=True, inplace=True)
extracted_text_df.head()

,Page No.,Heading,Page_Text,Text_Length,Metadata
0,Page 1,DOROTHEA GLAUSE S655,DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/...,30,"{'Section': 'DOROTHEA GLAUSE S655', 'Page_No.'..."
1,Page 3,POLICY RIDER,POLICY RIDER GROUP INSURANCE POLICY NO: S655 C...,230,"{'Section': 'POLICY RIDER', 'Page_No.': 'Page 3'}"
2,Page 5,PRINCIPAL LIFE INSURANCE COMPANY,PRINCIPAL LIFE INSURANCE COMPANY (called The P...,110,"{'Section': 'PRINCIPAL LIFE INSURANCE ', 'Page..."
3,Page 6,TABLE OF CONTENTS,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,153,"{'Section': 'TABLE OF CONTENTS', 'Page_No.': '..."
4,Page 7,Section A – Eligibility,Section A – Eligibility Member Life Insurance ...,176,"{'Section': 'Section A – Eligibility', 'Page_N..."


In [17]:
# Import the OpenAI Embedding Function into chroma

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [18]:
# Define the path where chroma collections will be stored

chroma_data_path = '/content/drive/MyDrive/Colab Notebooks/insuranceDb'

In [19]:
# Call PersistentClient()

client = chromadb.PersistentClient()

In [20]:
# Set up the embedding function using the OpenAI embedding model
openai_api_key = userdata.get('OPENAI_API_KEY')
os.environ["OPENAI_API_KEY"] = openai_api_key
model = "text-embedding-ada-002"
embedding_function = OpenAIEmbeddingFunction(api_key=openai_api_key, model_name=model)

In [21]:
# Initialise a collection in chroma and pass the embedding_function to it so that it used OpenAI embeddings to embed the documents
insurance_collection = client.get_or_create_collection(name='InsurancePolicyDoc', embedding_function=embedding_function)

In [22]:
documents_list = extracted_text_df["Page_Text"].tolist()
metadata_list =  extracted_text_df['Metadata'].tolist()

In [23]:
# Add the documents and metadata to the collection alongwith generic integer IDs. You can also feed the metadata information as IDs by combining the policy name and page no.

insurance_collection.add(
    documents= documents_list,
    ids = [str(i) for i in range(0, len(documents_list))],
    metadatas = metadata_list
)


In [24]:
# Let's take a look at the first few entries in the collection

insurance_collection.get(
    ids = ['0','1','2'],
    include = ['embeddings', 'documents', 'metadatas']
)

{'ids': ['0', '1', '2'],
 'embeddings': array([[-2.23880261e-02,  1.87088735e-02, -2.72935610e-02, ...,
         -3.68958823e-02,  2.89472216e-03, -1.34380336e-03],
        [-1.32057490e-02,  8.82212631e-03, -4.67860838e-03, ...,
         -1.56548154e-02, -4.84764605e-05,  7.25115696e-03],
        [-1.24035338e-02,  1.34377144e-02, -2.85228249e-03, ...,
         -2.97525711e-02, -1.01760682e-02,  9.71201342e-03]]),
 'documents': ['DOROTHEA GLAUSE S655 RHODE ISLAND JOHN DOE 01/01/2014 711 HIGH STREET GEORGE RI 02903 GROUP POLICY FOR: RHODE ISLAND JOHN DOE ALL MEMBERS Group Member Life Insurance Print Date: 07/16/2014',
  'POLICY RIDER GROUP INSURANCE POLICY NO: S655 COVERAGE: Life EMPLOYER: RHODE ISLAND JOHN DOE Effective on the later of the Date of Issue of this Group Policy or March 1, 2005, the following will apply to your Policy: From time to time The Principal may offer or provide certain employer groups who apply for coverage with The Principal a Financial Services Hotline and Gri

In [25]:
cache_collection = client.get_or_create_collection(name='Insurance_Cache', embedding_function=embedding_function)
cache_collection.peek()

{'ids': [],
 'embeddings': array([], dtype=float64),
 'documents': [],
 'uris': None,
 'included': ['metadatas', 'documents', 'embeddings'],
 'data': None,
 'metadatas': []}

#  Semantic Search with Cache

In [102]:
# Read the user query

query = input()

what are the  premium rate for each Member insured for Life Insurance ?


In [107]:
# Query the cache collection to check if the results are already stored
cache_results = cache_collection.query(
    query_texts=query,
    n_results=1
)

# Print the results from the cache query for debugging
cache_results



{'ids': [['what are the  premium rate for each Member insured for Life Insurance ?']],
 'embeddings': None,
 'documents': [['what are the  premium rate for each Member insured for Life Insurance ?']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'metadatas_0': '{"Page_No.": "Page 21", "Section": "b . on any date the defin"}',
    'distances_6': 0.336989164352417,
    'included_3': 'a',
    'documents_5': 'PART III - INDIVIDUAL REQUIREMENTS AND RIGHTS Section A - Eligibility Article 1 - Member Life Insurance A person will be eligible for Member Life Insurance on the date the person completes 30 consecutive days of continuous Active Work with the Policyholder as a Member. In no circumstance will a person be eligible for Member Life Insurance under this Group Policy if the person is eligible under any other Group Term Life Insurance policy underwritten by The Principal. Article 2 - Member Accidental Death and Dismemberment Insurance 

In [108]:
# If the cache did not return satisfactory results (e.g., based on distance), query the main collection
results = insurance_collection.query(
    query_texts=query,
    n_results=10
)

In [104]:
import json

threshold = 0.2

results_df_1 = pd.DataFrame()

# Query the cache collection to check if the results are already stored
cache_results = cache_collection.query(
    query_texts=query,
    n_results=1
)

# Print the results from the cache query for debugging
print(cache_results)

# Check if the cache is empty or if the distance exceeds the threshold
if not cache_results['distances'][0] or cache_results['distances'][0][0] > threshold:
    # Query the main collection for the top 10 results
    results = insurance_collection.query(
        query_texts=query,
        n_results=10
    )

    # Prepare keys and values for storing in cache
    cache_data = {}
    for key, val in results.items():
        if val is None:
            continue
        # Adjust the loop to match the actual number of items in val
        for i in range(min(len(val[0]), 10)):  # Ensure you only loop over existing items
            cache_data[f"{key}_{i}"] = val[0][i]

    # Flatten the metadata for storage in ChromaDB
    flat_cache_data = {}
    for k, v in cache_data.items():
        if isinstance(v, dict):
            # Convert the dictionary to a JSON string
            flat_cache_data[k] = json.dumps(v)
        else:
            flat_cache_data[k] = v

    # Store the query in cache
    cache_collection.add(
        documents=[query],
        ids=[query],  # Alternatively, you can use a unique ID
        metadatas=flat_cache_data
    )

    print("Not found in cache. Found in main collection.")

    # Convert the results to a DataFrame
    result_dict = {
        'Metadatas_1': results['metadatas'][0],
        'Documents_1': results['documents'][0],
        'Distances_1': results['distances'][0],
        'IDs': results['ids'][0]
    }
    results_df_1 = pd.DataFrame.from_dict(result_dict)

# If the distance is within the threshold, retrieve results from the cache
elif cache_results['distances'][0][0] <= threshold:
    # Extract data from the cache
    cache_result_dict = cache_results['metadatas'][0][0]
    ids = []
    documents = []
    distances = []
    metadatas = []

    # Collect data based on keys
    for key, value in cache_result_dict.items():
        if 'ids' in key:
            ids.append(value)
        elif 'documents' in key:
            documents.append(value)
        elif 'distances' in key:
            distances.append(value)
        elif 'metadatas' in key:
            metadatas.append(value)

    print("Found in cache!")

    # Convert the cache data to a DataFrame
    results_df_1 = pd.DataFrame({
        'IDs_1': ids,
        'Documents_1': documents,
        'Distances_1': distances,
        'Metadatas_1': metadatas
    })



{'ids': [['what are the  premium rate for each Member insured for Life Insurance ?']], 'embeddings': None, 'documents': [['what are the  premium rate for each Member insured for Life Insurance ?']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'ids_0': '18', 'distances_2': 0.3051954507827759, 'distances_9': 0.34077519178390503, 'ids_3': '43', 'metadatas_2': '{"Page_No.": "Page 20", "Section": "Section B - Premiums"}', 'ids_8': '5', 'ids_5': '23', 'distances_5': 0.3346160650253296, 'documents_6': "(1) only one Accelerated Benefit payment will be made during the Member's lifetime; and (2) the amount requested must be at least $5,000; and (3) in no event will payment exceed the lesser of: - 75% of the Member Life Insurance benefit in force on the date of the request; or - $250,000. The Accelerated Benefit payment will be made in a lump sum. d. Effect on Member Life Insurance Benefits If an Accelerated Benefit is paid, the Member Life Ins

In [29]:
# Display the DataFrame with results
results_df_1.head()

,Metadatas_1,Documents_1,Distances_1,IDs
0,"{'Page_No.': 'Page 21', 'Section': 'b . on any...",b . on any date the definition of Member or De...,0.271381,18
1,"{'Page_No.': 'Page 22', 'Section': 'The number...",The number of Members insured for Dependent Li...,0.291655,19
2,"{'Page_No.': 'Page 20', 'Section': 'Section B ...",Section B - Premiums Article 1 - Payment Respo...,0.305195,17
3,"{'Page_No.': 'Page 46', 'Section': 'PART IV - ...",PART IV - BENEFITS Section A - Member Life Ins...,0.318718,43
4,"{'Page_No.': 'Page 35', 'Section': 'Section C ...",Section C - Individual Terminations Article 1 ...,0.328987,32


In [33]:
query2=input()

what happens if i fail to pay to Premium?


In [34]:
cache_results= cache_collection.query(
         query_texts=query2,
          n_results=1
     )
cache_results

{'ids': [['what happens if i fail to Failure to Pay Premium ?']],
 'embeddings': None,
 'documents': [['what happens if i fail to Failure to Pay Premium ?']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'distances_3': 0.418071985244751,
    'documents_7': "The number of Members insured for Dependent Life Insurance will be multiplied by the premium rate then in effect. To ensure accurate premium calculations, the Policyholder is responsible for reporting to The Principal, the following information during the stated time periods: a. Members who are eligible to become insured are to be reported during the month prior to or during the month that coverage becomes effective. b. Members whose coverage has terminated are to be reported within a month of the date coverage terminated. c. Changes in Member insurance class are to be reported within a month of the date that the change in insurance class took place. If a Member is added or a p

In [35]:
threshold = 0.2

results_df_2 = pd.DataFrame()

# Query the cache collection to check if the results are already stored
cache_results = cache_collection.query(
    query_texts=query2,
    n_results=1
)

# Print the results from the cache query for debugging
print(cache_results)

# Check if the cache is empty or if the distance exceeds the threshold
if not cache_results['distances'][0] or cache_results['distances'][0][0] > threshold:
    # Query the main collection for the top 10 results
    results = insurance_collection.query(
        query_texts=query2,
        n_results=10
    )

    # Prepare keys and values for storing in cache
    cache_data = {}
    for key, val in results.items():
        if val is None:
            continue
        # Adjust the loop to match the actual number of items in val
        for i in range(min(len(val[0]), 10)):  # Ensure you only loop over existing items
            cache_data[f"{key}_{i}"] = val[0][i]

    # Flatten the metadata for storage in ChromaDB
    flat_cache_data = {}
    for k, v in cache_data.items():
        if isinstance(v, dict):
            # Convert the dictionary to a JSON string
            flat_cache_data[k] = json.dumps(v)
        else:
            flat_cache_data[k] = v

    # Store the query in cache
    cache_collection.add(
        documents=[query2],
        ids=[query2],  # Alternatively, you can use a unique ID
        metadatas=flat_cache_data
    )

    print("Not found in cache. Found in main collection.")

    # Convert the results to a DataFrame
    result_dict = {
        'Metadatas_2': results['metadatas'][0],
        'Documents_2': results['documents'][0],
        'Distances_2': results['distances'][0],
        'IDs': results['ids'][0]
    }
    results_df_2 = pd.DataFrame.from_dict(result_dict)

# If the distance is within the threshold, retrieve results from the cache
elif cache_results['distances'][0][0] <= threshold:
    # Extract data from the cache
    cache_result_dict = cache_results['metadatas'][0][0]
    ids = []
    documents = []
    distances = []
    metadatas = []

    # Collect data based on keys
    for key, value in cache_result_dict.items():
        if 'ids' in key:
            ids.append(value)
        elif 'documents' in key:
            documents.append(value)
        elif 'distances' in key:
            distances.append(value)
        elif 'metadatas' in key:
            metadatas.append(value)

    print("Found in cache!")

    # Convert the cache data to a DataFrame
    results_df_2 = pd.DataFrame({
        'IDs_2': ids,
        'Documents_2': documents,
        'Distances_2': distances,
        'Metadatas_2': metadatas
    })

{'ids': [['what happens if i fail to Failure to Pay Premium ?']], 'embeddings': None, 'documents': [['what happens if i fail to Failure to Pay Premium ?']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'ids_8': '44', 'distances_2': 0.4021034836769104, 'included_5': 'a', 'documents_2': "T he Principal may terminate the Policyholder's coverage on any premium due date if the Policyholder relocates to a state where this Group Policy is not marketed, by giving the Policyholder 31 days advanced notice in Writing. Article 4 - Policyholder Responsibility to Members If this Group Policy terminates for any reason, the Policyholder must: a. notify each Member of the effective date of the termination; and b. refund or otherwise account to each Member all contributions received or withheld from Members for premiums not actually paid to The Principal. This policy has been updated effective January 1, 2014 PART II - POLICY ADMINISTRATION GC 6005 Sec

In [36]:
results_df_2.head()

,IDs_2,Documents_2,Distances_2,Metadatas_2
0,44,T he Principal may terminate the Policyholder'...,0.402103,"{""Page_No."": ""Page 21"", ""Section"": ""b . on any..."
1,18,The number of Members insured for Dependent Li...,0.436278,"{""Section"": ""a. be actively engaged in"", ""Page..."
2,51,"M ember's death, the Death Benefits Payable ma...",0.418072,"{""Page_No."": ""Page 47"", ""Section"": ""M ember's ..."
3,20,Section C - Policy Termination Article 1 - Fai...,0.388825,"{""Section"": ""Section C - Policy Termin"", ""Page..."
4,17,c . a copy of the form which contains the stat...,0.443871,"{""Section"": ""f . claim requirements li"", ""Page..."


In [43]:
query3 = input()

what documentaion are required fpr filing a claim?


In [100]:
cache_results= cache_collection.query(
         query_texts=query3,
          n_results=1
     )
cache_results

{'ids': [['what documentaion are required fpr filing a claim?']],
 'embeddings': None,
 'documents': [['what documentaion are required fpr filing a claim?']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'metadatas_3': '{"Page_No.": "Page 54", "Section": "f . claim requirements li"}',
    'ids_3': '51',
    'ids_5': '12',
    'distances_5': 0.42379266023635864,
    'documents_0': 'Section D - Claim Procedures Article 1 - Notice of Claim Written notice must be sent to The Principal by or for a Member or Dependent who wishes to file claim for benefits under this Group Policy. This notice must be sent within 20 days after the date of the loss for which claim is being made. Failure to give notice within the time specified will not invalidate or reduce any claim if notice is given as soon as reasonably possible. Article 2 - Claim Forms The Principal, when it receives notice of claim, will provide appropriate claim forms for filing proo

In [45]:
threshold = 0.2

results_df_3 = pd.DataFrame()

# Query the cache collection to check if the results are already stored
cache_results = cache_collection.query(
    query_texts=query3,
    n_results=1
)

# Print the results from the cache query for debugging
#print(cache_results)

# Check if the cache is empty or if the distance exceeds the threshold
if not cache_results['distances'][0] or cache_results['distances'][0][0] > threshold:
    # Query the main collection for the top 10 results
    results = insurance_collection.query(
        query_texts=query3,
        n_results=10
    )

    # Prepare keys and values for storing in cache
    cache_data = {}
    for key, val in results.items():
        if val is None:
            continue
        # Adjust the loop to match the actual number of items in val
        for i in range(min(len(val[0]), 10)):  # Ensure you only loop over existing items
            cache_data[f"{key}_{i}"] = val[0][i]

    # Flatten the metadata for storage in ChromaDB
    flat_cache_data = {}
    for k, v in cache_data.items():
        if isinstance(v, dict):
            # Convert the dictionary to a JSON string
            flat_cache_data[k] = json.dumps(v)
        else:
            flat_cache_data[k] = v

    # Store the query in cache
    cache_collection.add(
        documents=[query3],
        ids=[query3],  # Alternatively, you can use a unique ID
        metadatas=flat_cache_data
    )

    print("Not found in cache. Found in main collection.")

    # Convert the results to a DataFrame
    result_dict = {
        'Metadatas_3': results['metadatas'][0],
        'Documents_3': results['documents'][0],
        'Distances_3': results['distances'][0],
        'IDs': results['ids'][0]
    }
    results_df_3 = pd.DataFrame.from_dict(result_dict)

# If the distance is within the threshold, retrieve results from the cache
elif cache_results['distances'][0][0] <= threshold:
    # Extract data from the cache
    cache_result_dict = cache_results['metadatas'][0][0]
    ids = []
    documents = []
    distances = []
    metadatas = []

    # Collect data based on keys
    for key, value in cache_result_dict.items():
        if 'ids' in key:
            ids.append(value)
        elif 'documents' in key:
            documents.append(value)
        elif 'distances' in key:
            distances.append(value)
        elif 'metadatas' in key:
            metadatas.append(value)

    print("Found in cache!")

    # Convert the cache data to a DataFrame
    results_df_3 = pd.DataFrame({
        'IDs_3': ids,
        'Documents_3': documents,
        'Distances_3': distances,
        'Metadatas_3': metadatas
    })

Not found in cache. Found in main collection.


In [46]:
results_df_3.head()

,Metadatas_3,Documents_3,Distances_3,IDs
0,"{'Page_No.': 'Page 61', 'Section': 'Section D ...",Section D - Claim Procedures Article 1 - Notic...,0.343139,58
1,"{'Section': 'A claimant may request an', 'Page...",A claimant may request an appeal of a claim de...,0.362038,59
2,"{'Section': 'c . a copy of the form wh', 'Page...",c . a copy of the form which contains the stat...,0.383302,15
3,"{'Page_No.': 'Page 54', 'Section': 'f . claim ...","f . claim requirements listed in PART IV, Sect...",0.396981,51
4,"{'Page_No.': 'Page 17', 'Section': 'a. be acti...",a. be actively engaged in business for profit ...,0.414800,14


In [47]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [48]:
cross_inputs_1 = [[query, response] for response in results_df_1['Documents_1']]
cross_rerank_scores_1 = cross_encoder.predict(cross_inputs_1)
cross_rerank_scores_1

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


array([ 2.2175784 ,  2.3985848 ,  1.9360971 ,  0.15906563, -2.4783213 ,
       -4.113749  ,  0.7123252 , -1.6021887 , -4.5288877 , -0.08958972],
      dtype=float32)

In [49]:
results_df_1['Reranked_scores'] = cross_rerank_scores_1
results_df_1

,Metadatas_1,Documents_1,Distances_1,IDs,Reranked_scores
0,"{'Page_No.': 'Page 21', 'Section': 'b . on any...",b . on any date the definition of Member or De...,0.271381,18,2.217578
1,"{'Page_No.': 'Page 22', 'Section': 'The number...",The number of Members insured for Dependent Li...,0.291655,19,2.398585
2,"{'Page_No.': 'Page 20', 'Section': 'Section B ...",Section B - Premiums Article 1 - Payment Respo...,0.305195,17,1.936097
3,"{'Page_No.': 'Page 46', 'Section': 'PART IV - ...",PART IV - BENEFITS Section A - Member Life Ins...,0.318718,43,0.159066
4,"{'Page_No.': 'Page 35', 'Section': 'Section C ...",Section C - Individual Terminations Article 1 ...,0.328987,32,-2.478321
5,"{'Page_No.': 'Page 26', 'Section': 'PART III -...",PART III - INDIVIDUAL REQUIREMENTS AND RIGHTS ...,0.334616,23,-4.113749
6,"{'Page_No.': 'Page 52', 'Section': '(1) only o...",(1) only one Accelerated Benefit payment will ...,0.336989,49,0.712325
7,"{'Page_No.': 'Page 59', 'Section': 'Section C ...",Section C - Dependent Life Insurance Article 1...,0.337550,56,-1.602189
8,"{'Section': 'Section A - Member Life I', 'Page...",Section A - Member Life Insurance Schedule of ...,0.337759,5,-4.528888
9,"{'Page_No.': 'Page 42', 'Section': 'Section F ...",Section F - Individual Purchase Rights Article...,0.340775,39,-0.089590


In [69]:
top_3_semantic_1 = results_df_1.sort_values(by='Distances_1')
top_3_semantic_1[:3]

,Metadatas_1,Documents_1,Distances_1,IDs,Reranked_scores
0,"{'Page_No.': 'Page 21', 'Section': 'b . on any...",b . on any date the definition of Member or De...,0.271381,18,2.217578
1,"{'Page_No.': 'Page 22', 'Section': 'The number...",The number of Members insured for Dependent Li...,0.291655,19,2.398585
2,"{'Page_No.': 'Page 20', 'Section': 'Section B ...",Section B - Premiums Article 1 - Payment Respo...,0.305195,17,1.936097


In [70]:
top_3_rerank_1 = results_df_1.sort_values(by='Reranked_scores', ascending=False)
top_3_rerank_1[:3]


,Metadatas_1,Documents_1,Distances_1,IDs,Reranked_scores
1,"{'Page_No.': 'Page 22', 'Section': 'The number...",The number of Members insured for Dependent Li...,0.291655,19,2.398585
0,"{'Page_No.': 'Page 21', 'Section': 'b . on any...",b . on any date the definition of Member or De...,0.271381,18,2.217578
2,"{'Page_No.': 'Page 20', 'Section': 'Section B ...",Section B - Premiums Article 1 - Payment Respo...,0.305195,17,1.936097


In [71]:
print(top_3_rerank_1[:3])

                                         Metadatas_1  \
1  {'Page_No.': 'Page 22', 'Section': 'The number...   
0  {'Page_No.': 'Page 21', 'Section': 'b . on any...   
2  {'Page_No.': 'Page 20', 'Section': 'Section B ...   

                                         Documents_1  Distances_1 IDs  \
1  The number of Members insured for Dependent Li...     0.291655  19   
0  b . on any date the definition of Member or De...     0.271381  18   
2  Section B - Premiums Article 1 - Payment Respo...     0.305195  17   

   Reranked_scores  
1         2.398585  
0         2.217578  
2         1.936097  


In [72]:
cross_inputs_2 = [[query2, response] for response in results_df_2['Documents_2']]
cross_rerank_scores_2 = cross_encoder.predict(cross_inputs_2)
results_df_2['Reranked_scores'] = cross_rerank_scores_2

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [73]:
top_3_semantic_2 = results_df_2.sort_values(by='Distances_2')
top_3_semantic_2[:3]

,IDs_2,Documents_2,Distances_2,Metadatas_2,Reranked_scores
5,19,a. be actively engaged in business for profit ...,0.351263,"{""Page_No."": ""Page 22"", ""Section"": ""The number...",-8.713665
3,20,Section C - Policy Termination Article 1 - Fai...,0.388825,"{""Section"": ""Section C - Policy Termin"", ""Page...",3.999949
0,44,T he Principal may terminate the Policyholder'...,0.402103,"{""Page_No."": ""Page 21"", ""Section"": ""b . on any...",-3.041489


In [74]:
top_3_rerank_2 = results_df_2.sort_values(by='Reranked_scores', ascending=False)
top_3_rerank_2[:3]

,IDs_2,Documents_2,Distances_2,Metadatas_2,Reranked_scores
3,20,Section C - Policy Termination Article 1 - Fai...,0.388825,"{""Section"": ""Section C - Policy Termin"", ""Page...",3.999949
0,44,T he Principal may terminate the Policyholder'...,0.402103,"{""Page_No."": ""Page 21"", ""Section"": ""b . on any...",-3.041489
6,21,TABLE OF CONTENTS PART I - DEFINITIONS PART II...,0.435528,"{""Page_No."": ""Page 24"", ""Section"": ""T he Princ...",-4.321020


In [75]:
cross_inputs_3 = [[query3, response] for response in results_df_3['Documents_3']]
cross_rerank_scores_3 = cross_encoder.predict(cross_inputs_3)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [76]:
results_df_3['Reranked_scores'] = cross_rerank_scores_3
top_3_semantic_3 = results_df_3.sort_values(by='Distances_3')
top_3_semantic_3[:3]

,Metadatas_3,Documents_3,Distances_3,IDs,Reranked_scores
0,"{'Page_No.': 'Page 61', 'Section': 'Section D ...",Section D - Claim Procedures Article 1 - Notic...,0.343139,58,-2.471166
1,"{'Section': 'A claimant may request an', 'Page...",A claimant may request an appeal of a claim de...,0.362038,59,-4.036301
2,"{'Section': 'c . a copy of the form wh', 'Page...",c . a copy of the form which contains the stat...,0.383302,15,-10.063317


In [77]:
top_3_rerank_3 = results_df_3.sort_values(by='Reranked_scores', ascending=False)
top_3_rerank_3[:3]

,Metadatas_3,Documents_3,Distances_3,IDs,Reranked_scores
0,"{'Page_No.': 'Page 61', 'Section': 'Section D ...",Section D - Claim Procedures Article 1 - Notic...,0.343139,58,-2.471166
3,"{'Page_No.': 'Page 54', 'Section': 'f . claim ...","f . claim requirements listed in PART IV, Sect...",0.396981,51,-2.598758
1,"{'Section': 'A claimant may request an', 'Page...",A claimant may request an appeal of a claim de...,0.362038,59,-4.036301


In [78]:
top_3_RAG_1 = top_3_rerank_1[["Documents_1", "Metadatas_1"]][:3]
top_3_RAG_2 = top_3_rerank_2[["Documents_2", "Metadatas_2"]][:3]
top_3_RAG_3 = top_3_rerank_3[["Documents_3", "Metadatas_3"]][:3]

In [79]:
top_3_RAG_1

,Documents_1,Metadatas_1
1,The number of Members insured for Dependent Li...,"{'Page_No.': 'Page 22', 'Section': 'The number..."
0,b . on any date the definition of Member or De...,"{'Page_No.': 'Page 21', 'Section': 'b . on any..."
2,Section B - Premiums Article 1 - Payment Respo...,"{'Page_No.': 'Page 20', 'Section': 'Section B ..."


In [80]:
# Define the function to generate the response. Provide a comprehensive prompt that passes the user query and the top 3 results to the model

def generate_response(query, top_3_RAG):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
        {"role": "system", "content": "You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents."},
        {"role": "user", "content": f"""
            You are a helpful assistant in the insurance domain who can effectively answer user queries about insurance policies and documents.
            You have a question asked by the user in '{query}' and you have some search results from a corpus of insurance documents in the dataframe '{top_3_RAG}'. These search results are essentially one page of an insurance document that may be relevant to the user query.

            The column 'documents' inside this dataframe contains the actual text from the policy document and the column 'metadata' contains the policy name and source page. The text inside the document may also contain tables in the format of a list of lists where each of the nested lists indicates a row.

            Use the documents in '{top_3_RAG}' to answer the query '{query}'. Frame an informative answer and also, use the dataframe to return the relevant policy names and page numbers as citations.

            Follow the guidelines below when performing the task:
            1. Try to provide relevant/accurate numbers if available.
            2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
            3. If the document text has tables with relevant information, please reformat the table and return the final information in a tabular format.
            4. Use the 'metadata' columns in the dataframe to retrieve and cite the policy name(s) and page number(s) as citation.
            5. If you can't provide the complete answer, please also provide any information that will help the user to search specific sections in the relevant cited documents.
            6. You are a customer-facing assistant, so do not provide any information on internal workings, just answer the query directly.

            The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text along with the citation. Provide your complete response first with all information, and then provide the citations.

            ### Few-Shot Examples

            ### Example 1: Basic Query about Coverage
            **Query:**
            What does the policy say about coverage for accidental death?

            **Top 3 RAG Results:**
            - **Document 1:** "This policy provides coverage for accidental death. The insured amount for accidental death is 200% of the base coverage amount if the death occurs within 90 days of the accident..."
            - **Document 2:** "Accidental death benefits are payable under this policy if the insured dies as a result of an accident. The benefit amount equals double the coverage amount, provided the death is a direct result of the accident and occurs within a specified time frame..."
            - **Document 3:** "In the event of accidental death, the policy pays an additional benefit, which is equal to twice the original coverage amount. This benefit is contingent on the death occurring within 180 days from the date of the accident..."

            **Response:**
            The policy provides coverage for accidental death, where the benefit amount is typically 200% of the base coverage. The death must occur as a direct result of an accident and within a specified period, which varies between 90 to 180 days depending on the policy.
            **Citations:**
            Document 1: Heading/Section X, Page 5
            Document 2: Heading/Section Y, Page 12
            Document 3: Heading/Section Z, Page 7

            ### Example 2: Query about Exclusions
            **Query:**
            Are there any exclusions for pre-existing conditions in this policy?

            **Top 3 RAG Results:**
            - **Document 1:** "This policy excludes coverage for any conditions that were diagnosed or treated within 12 months prior to the policy's start date. However, if the condition remains stable for 24 months after the policy's start date, it may be eligible for coverage..."
            - **Document 2:** "Pre-existing conditions are generally not covered under this policy unless explicitly stated otherwise. Any condition that has shown symptoms or required medical attention in the 12 months before the policy start date is excluded..."
            - **Document 3:** "Exclusions apply to pre-existing conditions if they were present within a 12-month window before the policy commencement. After a waiting period of 24 months, these conditions may be reconsidered for coverage..."

            **Response:**
            The policy excludes coverage for pre-existing conditions that were diagnosed or treated within 12 months before the policy start date. However, if the condition remains stable and does not require treatment for 24 months after the policy start date, it may be eligible for coverage.
            **Citations:**
            Document 1: Heading/Section X, Page 8
            Document 2: Heading/Section Y, Page 15
            Document 3: Heading/Section Z, Page 10

            ### Example 3: Query about Beneficiaries
            **Query:**
            How can I update the beneficiary for my life insurance policy?

            **Top 3 RAG Results:**
            - **Document 1:** "To update the beneficiary, the policyholder must submit a written request to the insurance company. The request should include the policy number, current beneficiary, and the new beneficiary's details..."
            - **Document 2:** "Beneficiary changes can be made by filling out the 'Beneficiary Change Form,' which must be signed by the policyholder and submitted to the insurance provider. The change takes effect upon the company's receipt and acceptance of the form..."
            - **Document 3:** "The policyholder has the right to change the beneficiary at any time by providing written notice to the insurer. The change will be recorded and acknowledged by the insurer upon receipt..."

            **Response:**
            To update the beneficiary of your life insurance policy, you need to submit a written request or complete a 'Beneficiary Change Form' provided by your insurer. This request must be signed by you as the policyholder, and the change will take effect once the insurance company has received and acknowledged it.
            **Citations:**
            Document 1: Heading/Section X, Page 3
            Document 2: Heading/Section Y, Page 6
            Document 3: Heading/Section Z, Page 4

            ### Example 4: Complex Query about Premium Payments
            **Query:**
            What are the options if I miss a premium payment?

            **Top 3 RAG Results:**
            - **Document 1:** "If a premium payment is missed, the policyholder typically has a 30-day grace period to make the payment without penalty. If payment is not received within this period, the policy may lapse, and coverage could be lost..."
            - **Document 2:** "The policy includes a 30-day grace period for missed payments. During this time, coverage remains in force. If payment is not made by the end of the grace period, the policyholder may request reinstatement of the policy, subject to underwriting..."
            - **Document 3:** "In the event of a missed premium, a 30-day grace period is granted. If the premium is not paid within this period, the policyholder may choose to reinstate the policy, which may require proof of insurability and payment of overdue premiums..."

            **Response:**
            If you miss a premium payment, your policy provides a 30-day grace period during which you can make the payment without losing coverage. If the payment is not made within this period, the policy may lapse. However, you may have the option to reinstate the policy by providing proof of insurability and paying the overdue premiums.
            **Citations:**
            Document 1: Heading/Section X, Page 10
            Document 2: Heading/Section Y, Page 11
            Document 3: Heading/Section Z, Page 9
        """},
    ]

    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return response.choices[0].message.content.split('\n')

In [81]:
response = generate_response(query, top_3_RAG_1)

In [84]:
print("\n".join(response))

The documents reviewed did not directly provide specific premium rates for each member insured under the life insurance policy. However, there is information regarding the general responsibility for premium payments and the definitions involved in determining the insured members.

To find specific premium rates for each member insured for life insurance, I recommend referring to the section titled "Section B - Premiums," where detailed information regarding premium responsibilities is outlined. This section likely contains the necessary rates or criteria for calculating premiums based on member classifications.

**Citations:**
- Document 2: Section B - Premiums, Page 20
- Document 1: The number of Members insured for Dependent Life Insurance, Page 22


In [85]:
response = generate_response(query2, top_3_RAG_2)

In [86]:
print("\n".join(response))

If you fail to pay your premium, most insurance policies provide a grace period—typically 30 days—during which you can make the payment without penalties. If you do not pay within this grace period, the policy may lapse, resulting in the loss of coverage. Once the policy is lapsed, you may be able to request reinstatement; however, this process might involve proof of insurability and payment of any overdue premiums.

For detailed terms related to policy termination due to non-payment of premium, it is advisable to refer to specific sections and articles in your policy documents.

**Citations:**  
Document 1: Section C - Policy Termination, Page 21  
Document 2: Relevant - Page 24


In [87]:
response = generate_response(query3, top_3_RAG_3)

In [88]:
print("\n".join(response))

To file a claim, certain documentation is typically required to ensure the process is smooth and efficient. Below are the essential documents that you may need to prepare for submitting a claim:

1. **Claim Form**: This is a formal document that needs to be completed and submitted along with the supporting documents.
   
2. **Proof of Loss**: You may need to provide documentation that substantiates your loss, such as receipts, invoices, or photographs when applicable.

3. **Policy Information**: Include a copy of your insurance policy or policy number, which will help the insurer correlate your claim with coverage.

4. **Additional Documentation**: Depending on the nature of the claim, you may be required to submit specific documents such as medical reports, death certificates, or police reports.

5. **Identification**: A valid form of identification may also be necessary to verify your identity.

Based on the documents reviewed, the relevant sections provide detailed procedures on fil

# Final Modular Code

In [89]:
def get_context(query):
 threshold = 0.2

 results_df = pd.DataFrame()

 # Query the cache collection to check if the results are already stored
 cache_results = cache_collection.query(
    query_texts=query,
    n_results=1
 )

 # Print the results from the cache query for debugging
 print(cache_results)

 # Check if the cache is empty or if the distance exceeds the threshold
 if not cache_results['distances'][0] or cache_results['distances'][0][0] > threshold:
    # Query the main collection for the top 10 results
    results = insurance_collection.query(
        query_texts=query,
        n_results=10
    )

    # Prepare keys and values for storing in cache
    cache_data = {}
    for key, val in results.items():
        if val is None:
            continue
        # Adjust the loop to match the actual number of items in val
        for i in range(min(len(val[0]), 10)):  # Ensure you only loop over existing items
            cache_data[f"{key}_{i}"] = val[0][i]

    # Flatten the metadata for storage in ChromaDB
    flat_cache_data = {}
    for k, v in cache_data.items():
        if isinstance(v, dict):
            # Convert the dictionary to a JSON string
            flat_cache_data[k] = json.dumps(v)
        else:
            flat_cache_data[k] = v

    # Store the query in cache
    cache_collection.add(
        documents=[query],
        ids=[query],  # Alternatively, you can use a unique ID
        metadatas=flat_cache_data
    )

    print("Not found in cache. Found in main collection.")

    # Convert the results to a DataFrame
    result_dict = {
        'Metadatas': results['metadatas'][0],
        'Documents': results['documents'][0],
        'Distances': results['distances'][0],
        'IDs': results['ids'][0]
    }
    results_df = pd.DataFrame.from_dict(result_dict)

 # If the distance is within the threshold, retrieve results from the cache
 elif cache_results['distances'][0][0] <= threshold:
    # Extract data from the cache
    cache_result_dict = cache_results['metadatas'][0][0]
    ids = []
    documents = []
    distances = []
    metadatas = []

    # Collect data based on keys
    for key, value in cache_result_dict.items():
        if 'ids' in key:
            ids.append(value)
        elif 'documents' in key:
            documents.append(value)
        elif 'distances' in key:
            distances.append(value)
        elif 'metadatas' in key:
            metadatas.append(value)

    print("Found in cache!")

    # Convert the cache data to a DataFrame
    results_df = pd.DataFrame({
        'IDs': ids,
        'Documents': documents,
        'Distances': distances,
        'Metadatas': metadatas
    })
    return results_df

In [90]:
def rerank(results_df):
  cross_inputs = [[query, response] for response in results_df['Documents']]
  cross_rerank_scores = cross_encoder.predict(cross_inputs)
  results_df['Reranked_scores'] = cross_rerank_scores
  return results_df

In [91]:
def top_3_context(results_df):
  top_3_rerank = results_df.sort_values(by='Reranked_scores', ascending=False)
  return top_3_rerank[:3]

In [94]:
def get_reply(query):
    results_df = get_context(query)
    results_df = rerank(results_df)
    top_3_rerank = top_3_context(results_df)
    response=generate_response(query, top_3_rerank)
    return "\n".join(response)

In [98]:
query = input()

what happens if i fail to pay to Premium?


In [99]:
print(get_reply(query))

{'ids': [['what happens if i fail to Failure to Pay Premium ?']], 'embeddings': None, 'documents': [['what happens if i fail to Failure to Pay Premium ?']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'ids_1': '17', 'documents_7': "The number of Members insured for Dependent Life Insurance will be multiplied by the premium rate then in effect. To ensure accurate premium calculations, the Policyholder is responsible for reporting to The Principal, the following information during the stated time periods: a. Members who are eligible to become insured are to be reported during the month prior to or during the month that coverage becomes effective. b. Members whose coverage has terminated are to be reported within a month of the date coverage terminated. c. Changes in Member insurance class are to be reported within a month of the date that the change in insurance class took place. If a Member is added or a present Member's insurance is 

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


If you fail to pay your premium, your policy typically includes a grace period during which you can still make the payment without any penalties. For most policies, this grace period is commonly 30 days. During this time, your coverage remains active. However, if you do not make the payment within the grace period, the policy may lapse, resulting in the loss of coverage. 

If your policy has lapsed due to non-payment, you may have the option to reinstate it. This often requires submitting proof of insurability and paying any overdue premiums. It's important to check the specific terms laid out in your policy document regarding missed payments and reinstatement procedures, as they can vary.

**Citations:**  
Document 1: Section C - Policy Termination Article 1 - Failure to Pay, Page 47  
Document 2: Section 3 - Policyholder Obligations, Page 12  
Document 3: Section 4 - Member's Responsibilities, Page 18
